In [1]:
import numpy as np
import math
import copy

gamma = 0.6
N_e = 2

In [2]:
class State:
    ball_x = 0
    ball_y = 0
    velocity_x = 0
    velocity_y = 0
    paddle_y = 0
    terminate = False
    
    
    def print_state(self):
        print(np.around(self.ball_x, 2) , " " , np.around(self.ball_y, 2) , " " ,  
              np.around(self.velocity_x, 2) , " " ,  np.around(self.velocity_y, 2) , " " ,  np.around(self.paddle_y, 2))
              
 
    def __init__(self, bx, by, vx, vy, p):
        self.ball_x = bx
        self.ball_y = by
        self.velocity_x = vx
        self.velocity_y = vy
        self.paddle_y = p
        
         
    def up(self):
        self.paddle_y = max(self.paddle_y - 0.04, 0)
        
        self.update()
        #print("in function up", match(self))

    def down(self):
        self.paddle_y = min(self.paddle_y + 0.04, 0.8)
        self.update()

    def update(self):
        global num_bounce
#         global state_tracker
        
        self.ball_x += self.velocity_x
        self.ball_y += self.velocity_y
        
        if self.ball_x >= 1:        # hitting right bound
            
            
            if self.ball_y >= self.paddle_y and self.ball_y <= self.paddle_y + 0.2:    # hitting paddle
                self.ball_x = 2 - self.ball_x
                #print("bounce")
                num_bounce += 1
#                 if curr_state not in state_tracker:
#                     #print(curr_state)
#                     state_tracker.append(curr_state)
                                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
                    
                
                state_tracker = []
                
            else:    # missing paddle
                #print("terminate")
                self.ball_x = 2 - self.ball_x
#                 reward(state_tracker,-1)
#                 state_tracker = []
                self.terminate = True

            
        if self.ball_x <= 0:     # hitting left bound
            self.ball_x = -self.ball_x
            self.velocity_x = -self.velocity_x
        if self.ball_y <= 0:     # hitting upper bound
            self.ball_y = -self.ball_y
            self.velocity_y = -self.velocity_y
        if self.ball_y  >= 1:    # hitting right bound
            self.ball_y = 2 - self.ball_y
            self.velocity_y = -self.velocity_y

In [3]:
def match(s):
    b_x = math.floor(s.ball_x * 12)
    b_y = math.floor(s.ball_y * 12)
    if s.velocity_x > 0:
        v_x = 1
    else:
        v_x = 0    # velocity_x <= 0 map to 0

    v_y = 0
    if s.velocity_y > 0 and abs(s.velocity_y) > 0.015:
        v_y = 1
    elif s.velocity_y < 0 and abs(s.velocity_y) > 0.015:    #velocity_y < 0.015 map to 2
        v_y = 2

    #print("paddle_y", s.paddle_y)
    p = math.floor(s.paddle_y / (1 - 0.2) * 12)
    if p == 12:
        p = 11
    #print("p", p)

    return (b_x ,b_y, v_x, v_y, p)

In [4]:
def reward(state):
    next_ball_x = state.ball_x + state.velocity_x

    if next_ball_x >= 1:        # hitting right bound
        if state.ball_y >= state.paddle_y and state.ball_y <= state.paddle_y + 0.2:    # hitting paddle
            return 1
        else:     # missing paddle
            return -1
    return 0

In [5]:

def get_up(current_state):
    #print("up current", match(current_state))
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.up()
    #print("new state", match(new_state))
    return new_state

def get_down(current_state):
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.down()
    return new_state

def get_no_move(current_state):
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.update()
    return new_state

In [6]:
def max_Q(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    Q_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    #print(b_x ,b_y, v_x, v_y, p)
    Q_down = Q_table[b_x][b_y][v_x][v_y][p][1]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    Q_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    #print(Q_up, Q_down, Q_no_move)
    
    return max(Q_up, Q_down, Q_no_move)

In [7]:
def argmax_f(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][0] < N_e:
        f_up = 0
    else:
        f_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][1] < N_e:
        f_down = 0
    else:
        f_down = Q_table[b_x][b_y][v_x][v_y][p][1]
        
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][2] < N_e:
        f_no_move = 0
    else:
        f_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    if f_up == f_down and f_down == f_no_move:
        return np.random.randint(3)
    
    return np.argmax([f_up, f_down, f_no_move])

In [8]:
def alpha(state):
    (b_x ,b_y, v_x, v_y, p) = match(state)
    num_visited = N_sa[b_x][b_y][v_x][v_y][p][a] 
    return 60 / (60 + num_visited)

In [9]:
Q_table = np.zeros((12, 12, 2, 3, 12, 3)) # x, y, v_x, v_y, paddle, action
Q_additional_state = -1                   # x >= 1 state
N_sa = np.zeros((12, 12, 2, 3, 12, 3))    # x, y, v_x, v_y, paddle, action

s = State(0,0,0,0,0)
s_empty = True
a = 0
r = 0

In [10]:
'''action: 0-up, 1-down, 2-no_move'''
def Q_agent(current_state, t):
    global s_empty
    global s
    global a
    global r
    global N_sa
    global Q
    if current_state.terminate:
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #Q_table[b_x][b_y][v_x][v_y][p][3] = reward(current_state)

    if s_empty is False:
        # increment N_s
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #print(b_x ,b_y, v_x, v_y, p, a)
        N_sa[b_x][b_y][v_x][v_y][p][a] += 1
        #print(N_sa[b_x][b_y][v_x][v_y][p][a])
        
        Q_table[b_x][b_y][v_x][v_y][p][a] =  Q_table[b_x][b_y][v_x][v_y][p][a] + \
                            alpha(s)* (r + gamma * max_Q(current_state) - Q_table[b_x][b_y][v_x][v_y][p][a])
        
    else:
        s_empty = False
    
    #s.empty = False
    s = copy.deepcopy(current_state)
    #print("s", match(s))
    
    (b_x ,b_y, v_x, v_y, p) = match(current_state)
    a = argmax_f(current_state)
    
    r = reward(current_state)
    
    return a

In [11]:

t_ = 1
num_bounce = 0
total_num_bounce = 0
for i in range(10000):
    s = State(0.5, 0.5, 0.03, 0.01, 0.4)
    current_state = copy.deepcopy(s)
    print("round: ", i)
    num_bounce = 0
    while s.terminate is False:
        #s.print_state()
        next_action = Q_agent(current_state, t_)
        #print("current state before action", match(current_state))
        #print("next action", next_action)
        if next_action == 0:
            current_state.up()
        if next_action == 1:
            current_state.down()
        if next_action == 2:
            current_state.update()
        #print("current state", match(current_state))
    t_ += 1
    print("bounce:", num_bounce)
    total_num_bounce += num_bounce
    print("state visited: ",np.count_nonzero(Q_table))
# print(total_num_bounce / 10000)

round:  0
bounce: 0
state visited:  1
round:  1
bounce: 7
state visited:  4
round:  2
bounce: 0
state visited:  5
round:  3
bounce: 0
state visited:  7
round:  4
bounce: 0
state visited:  9
round:  5
bounce: 0
state visited:  10
round:  6
bounce: 0
state visited:  11
round:  7
bounce: 0
state visited:  11
round:  8
bounce: 0
state visited:  12
round:  9
bounce: 0
state visited:  13
round:  10
bounce: 0
state visited:  14
round:  11
bounce: 7
state visited:  17
round:  12
bounce: 0
state visited:  18
round:  13
bounce: 4
state visited:  20
round:  14
bounce: 0
state visited:  22
round:  15
bounce: 0
state visited:  23
round:  16
bounce: 0
state visited:  24
round:  17
bounce: 5
state visited:  27
round:  18
bounce: 7
state visited:  30
round:  19
bounce: 11
state visited:  32
round:  20
bounce: 12
state visited:  35
round:  21
bounce: 7
state visited:  37
round:  22
bounce: 22
state visited:  41
round:  23
bounce: 0
state visited:  44
round:  24
bounce: 5
state visited:  46
round:  25
b

bounce: 7
state visited:  257
round:  217
bounce: 7
state visited:  262
round:  218
bounce: 4
state visited:  262
round:  219
bounce: 4
state visited:  262
round:  220
bounce: 12
state visited:  267
round:  221
bounce: 5
state visited:  268
round:  222
bounce: 5
state visited:  269
round:  223
bounce: 14
state visited:  274
round:  224
bounce: 14
state visited:  275
round:  225
bounce: 4
state visited:  275
round:  226
bounce: 5
state visited:  276
round:  227
bounce: 4
state visited:  276
round:  228
bounce: 4
state visited:  276
round:  229
bounce: 5
state visited:  277
round:  230
bounce: 5
state visited:  278
round:  231
bounce: 7
state visited:  279
round:  232
bounce: 5
state visited:  279
round:  233
bounce: 7
state visited:  279
round:  234
bounce: 14
state visited:  283
round:  235
bounce: 4
state visited:  283
round:  236
bounce: 4
state visited:  283
round:  237
bounce: 19
state visited:  292
round:  238
bounce: 4
state visited:  292
round:  239
bounce: 14
state visited:  29

bounce: 17
state visited:  681
round:  434
bounce: 19
state visited:  688
round:  435
bounce: 7
state visited:  689
round:  436
bounce: 11
state visited:  693
round:  437
bounce: 7
state visited:  694
round:  438
bounce: 5
state visited:  695
round:  439
bounce: 12
state visited:  697
round:  440
bounce: 5
state visited:  698
round:  441
bounce: 4
state visited:  698
round:  442
bounce: 5
state visited:  699
round:  443
bounce: 4
state visited:  699
round:  444
bounce: 5
state visited:  700
round:  445
bounce: 5
state visited:  703
round:  446
bounce: 7
state visited:  705
round:  447
bounce: 10
state visited:  707
round:  448
bounce: 26
state visited:  721
round:  449
bounce: 5
state visited:  723
round:  450
bounce: 5
state visited:  724
round:  451
bounce: 5
state visited:  725
round:  452
bounce: 7
state visited:  726
round:  453
bounce: 7
state visited:  727
round:  454
bounce: 14
state visited:  731
round:  455
bounce: 10
state visited:  735
round:  456
bounce: 4
state visited:  

bounce: 5
state visited:  1347
round:  637
bounce: 14
state visited:  1356
round:  638
bounce: 7
state visited:  1361
round:  639
bounce: 12
state visited:  1364
round:  640
bounce: 12
state visited:  1367
round:  641
bounce: 7
state visited:  1371
round:  642
bounce: 17
state visited:  1388
round:  643
bounce: 12
state visited:  1397
round:  644
bounce: 5
state visited:  1398
round:  645
bounce: 5
state visited:  1399
round:  646
bounce: 14
state visited:  1406
round:  647
bounce: 7
state visited:  1407
round:  648
bounce: 12
state visited:  1412
round:  649
bounce: 5
state visited:  1413
round:  650
bounce: 5
state visited:  1414
round:  651
bounce: 7
state visited:  1417
round:  652
bounce: 28
state visited:  1424
round:  653
bounce: 5
state visited:  1429
round:  654
bounce: 12
state visited:  1433
round:  655
bounce: 4
state visited:  1433
round:  656
bounce: 5
state visited:  1436
round:  657
bounce: 14
state visited:  1439
round:  658
bounce: 5
state visited:  1440
round:  659
b

bounce: 22
state visited:  2329
round:  841
bounce: 7
state visited:  2330
round:  842
bounce: 4
state visited:  2330
round:  843
bounce: 8
state visited:  2336
round:  844
bounce: 5
state visited:  2341
round:  845
bounce: 7
state visited:  2346
round:  846
bounce: 15
state visited:  2353
round:  847
bounce: 20
state visited:  2361
round:  848
bounce: 7
state visited:  2363
round:  849
bounce: 5
state visited:  2363
round:  850
bounce: 12
state visited:  2370
round:  851
bounce: 10
state visited:  2372
round:  852
bounce: 5
state visited:  2375
round:  853
bounce: 7
state visited:  2376
round:  854
bounce: 14
state visited:  2387
round:  855
bounce: 7
state visited:  2391
round:  856
bounce: 7
state visited:  2401
round:  857
bounce: 7
state visited:  2405
round:  858
bounce: 22
state visited:  2417
round:  859
bounce: 26
state visited:  2429
round:  860
bounce: 4
state visited:  2429
round:  861
bounce: 54
state visited:  2447
round:  862
bounce: 14
state visited:  2454
round:  863
b

...

round:  9268
bounce: 19
state visited:  14073
round:  9269
bounce: 16
state visited:  14073
round:  9270
bounce: 18
state visited:  14073
round:  9271
bounce: 11
state visited:  14073
round:  9272
bounce: 17
state visited:  14073
round:  9273
bounce: 56
state visited:  14073
round:  9274
bounce: 14
state visited:  14073
round:  9275
bounce: 38
state visited:  14073
round:  9276
bounce: 11
state visited:  14073
round:  9277
bounce: 19
state visited:  14073
round:  9278
bounce: 48
state visited:  14073
round:  9279
bounce: 12
state visited:  14073
round:  9280
bounce: 12
state visited:  14073
round:  9281
bounce: 23
state visited:  14073
round:  9282
bounce: 16
state visited:  14073
round:  9283
bounce: 14
state visited:  14073
round:  9284
bounce: 31
state visited:  14073
round:  9285
bounce: 11
state visited:  14073
round:  9286
bounce: 14
state visited:  14073
round:  9287
bounce: 25
state visited:  14073
round:  9288
bounce: 75
state visited:  14073
round:  9289
bounce: 50
state visi

bounce: 49
state visited:  14107
round:  9456
bounce: 26
state visited:  14107
round:  9457
bounce: 17
state visited:  14107
round:  9458
bounce: 11
state visited:  14107
round:  9459
bounce: 29
state visited:  14107
round:  9460
bounce: 16
state visited:  14107
round:  9461
bounce: 17
state visited:  14107
round:  9462
bounce: 14
state visited:  14107
round:  9463
bounce: 59
state visited:  14107
round:  9464
bounce: 9
state visited:  14107
round:  9465
bounce: 7
state visited:  14107
round:  9466
bounce: 29
state visited:  14107
round:  9467
bounce: 39
state visited:  14107
round:  9468
bounce: 40
state visited:  14107
round:  9469
bounce: 57
state visited:  14107
round:  9470
bounce: 9
state visited:  14107
round:  9471
bounce: 23
state visited:  14107
round:  9472
bounce: 35
state visited:  14107
round:  9473
bounce: 12
state visited:  14107
round:  9474
bounce: 14
state visited:  14107
round:  9475
bounce: 14
state visited:  14107
round:  9476
bounce: 12
state visited:  14107
roun

bounce: 28
state visited:  14143
round:  9635
bounce: 12
state visited:  14143
round:  9636
bounce: 13
state visited:  14143
round:  9637
bounce: 12
state visited:  14143
round:  9638
bounce: 47
state visited:  14143
round:  9639
bounce: 28
state visited:  14143
round:  9640
bounce: 20
state visited:  14143
round:  9641
bounce: 21
state visited:  14143
round:  9642
bounce: 28
state visited:  14144
round:  9643
bounce: 45
state visited:  14147
round:  9644
bounce: 14
state visited:  14147
round:  9645
bounce: 14
state visited:  14147
round:  9646
bounce: 66
state visited:  14147
round:  9647
bounce: 25
state visited:  14147
round:  9648
bounce: 25
state visited:  14147
round:  9649
bounce: 33
state visited:  14148
round:  9650
bounce: 48
state visited:  14148
round:  9651
bounce: 14
state visited:  14149
round:  9652
bounce: 28
state visited:  14149
round:  9653
bounce: 9
state visited:  14149
round:  9654
bounce: 23
state visited:  14149
round:  9655
bounce: 17
state visited:  14149
ro

bounce: 22
state visited:  14197
round:  9816
bounce: 44
state visited:  14197
round:  9817
bounce: 16
state visited:  14197
round:  9818
bounce: 15
state visited:  14197
round:  9819
bounce: 72
state visited:  14197
round:  9820
bounce: 47
state visited:  14197
round:  9821
bounce: 15
state visited:  14199
round:  9822
bounce: 23
state visited:  14199
round:  9823
bounce: 34
state visited:  14199
round:  9824
bounce: 80
state visited:  14199
round:  9825
bounce: 26
state visited:  14199
round:  9826
bounce: 87
state visited:  14199
round:  9827
bounce: 9
state visited:  14199
round:  9828
bounce: 64
state visited:  14200
round:  9829
bounce: 61
state visited:  14203
round:  9830
bounce: 20
state visited:  14203
round:  9831
bounce: 138
state visited:  14203
round:  9832
bounce: 33
state visited:  14203
round:  9833
bounce: 32
state visited:  14203
round:  9834
bounce: 58
state visited:  14203
round:  9835
bounce: 54
state visited:  14204
round:  9836
bounce: 43
state visited:  14204
r

bounce: 54
state visited:  14244
round:  9997
bounce: 28
state visited:  14244
round:  9998
bounce: 73
state visited:  14245
round:  9999
bounce: 28
state visited:  14245


In [12]:
print("average bouncing per game: ", total_num_bounce / 10000)

print("totoal state visited after training: ", np.count_nonzero(Q_table))



average bouncing per game:  26.931
totoal state visited after training:  14245


In [13]:
print (sorted(Q_table[np.nonzero(Q_table)],reverse=True), '\n')

[0.9984639147277713, 0.98887933093928759, 0.98361514941174932, 0.98360655737704916, 0.98360655737704916, 0.98360655737704916, 0.98009685212003772, 0.95111622827901043, 0.9390600313963986, 0.9369319821673926, 0.93367281120315404, 0.9249743204463704, 0.91537617342810651, 0.89632139181050086, 0.88055327413917095, 0.84896737572291836, 0.84543550834686021, 0.81545457997749371, 0.81501718260863287, 0.81134508811514405, 0.79039056081235859, 0.78096728204468024, 0.78092409625210446, 0.77773454733006664, 0.75358710803675377, 0.74227789861362203, 0.7202675340728677, 0.71569270829282083, 0.71321604625624868, 0.70907429561273672, 0.70040529664374129, 0.69596422140424641, 0.67534505500091946, 0.65626583813149142, 0.65075390774885944, 0.64292235997670277, 0.64115891154772686, 0.63824824417399473, 0.63452233579350481, 0.62680034569428478, 0.62085589196674529, 0.6203440271260322, 0.61788832360587398, 0.60974639379330164, 0.60176615256752086, 0.59685335372463566, 0.59517456873169938, 0.5901490729537300

In [14]:
print (sorted(N_sa[np.nonzero(N_sa)],reverse=True), '\n')

[18586.0, 18476.0, 14643.0, 13817.0, 13490.0, 9635.0, 9336.0, 9120.0, 9084.0, 8315.0, 7431.0, 6737.0, 6251.0, 5396.0, 4047.0, 3811.0, 3679.0, 3206.0, 3178.0, 3080.0, 3015.0, 3005.0, 2976.0, 2917.0, 2909.0, 2906.0, 2845.0, 2790.0, 2750.0, 2565.0, 2539.0, 2481.0, 2454.0, 2434.0, 2384.0, 2380.0, 2338.0, 2321.0, 2278.0, 2269.0, 2265.0, 2250.0, 2240.0, 2239.0, 2234.0, 2212.0, 2199.0, 2186.0, 2179.0, 2161.0, 2160.0, 2116.0, 2111.0, 2096.0, 2071.0, 2052.0, 2041.0, 2035.0, 2022.0, 2016.0, 2000.0, 1993.0, 1976.0, 1976.0, 1921.0, 1906.0, 1894.0, 1878.0, 1841.0, 1835.0, 1825.0, 1817.0, 1812.0, 1797.0, 1794.0, 1778.0, 1768.0, 1723.0, 1714.0, 1709.0, 1708.0, 1708.0, 1695.0, 1677.0, 1642.0, 1641.0, 1632.0, 1626.0, 1626.0, 1619.0, 1610.0, 1606.0, 1605.0, 1600.0, 1583.0, 1572.0, 1566.0, 1562.0, 1560.0, 1551.0, 1543.0, 1542.0, 1525.0, 1518.0, 1516.0, 1516.0, 1514.0, 1514.0, 1514.0, 1511.0, 1494.0, 1489.0, 1478.0, 1477.0, 1466.0, 1465.0, 1464.0, 1460.0, 1451.0, 1450.0, 1446.0, 1438.0, 1438.0, 1435.0, 14

In [15]:
# print (N_sa[np.nonzero(N_sa)])

import statistics as stat

In [16]:
data = N_sa[np.nonzero(N_sa)]

In [17]:
print (stat.mean(data), stat.median(data), stat.mode(data))

print (stat.median_low(data), stat.median_high(data))

print (stat.median_grouped(data))

print (stat.harmonic_mean(data))

93.4726352166 6.0 1.0
6.0 6.0
5.66612729234
3.28496920205


In [18]:
print(stat.stdev(data))

356.39813616136666


In [19]:
def Q_agent_test(current_state, t):
    global s_empty
    global s
    global a
    global r
    global N_sa
    global Q


    if s_empty is False:
        # increment N_s
        (b_x ,b_y, v_x, v_y, p) = match(s)
        
    else:
        s_empty = False
    
    #s.empty = False
    s = copy.deepcopy(current_state)
    #print("s", match(s))
    
    (b_x ,b_y, v_x, v_y, p) = match(current_state)
    a = argmax_f_test(current_state)
    
    r = reward(current_state)
    
    return a

In [20]:
def argmax_f_test(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up_test(current_state))
    #print ('up' , (b_x ,b_y, v_x, v_y, p))
    f_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down_test(current_state))
    #print ('down', (b_x ,b_y, v_x, v_y, p))

    f_down = Q_table[b_x][b_y][v_x][v_y][p][1]
        
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move_test(current_state))
    #print ('nothing',(b_x ,b_y, v_x, v_y, p))

    f_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    if f_up == f_down and f_down == f_no_move:
        return np.random.randint(3)
    
    return np.argmax([f_up, f_down, f_no_move])

In [21]:
print("average bouncing per game: ", total_num_bounce / 1000)

print("totoal state visited after training: ", np.count_nonzero(Q_table))

average bouncing per game:  269.31
totoal state visited after training:  14245


In [22]:
class enemy():
    position = 0.4
    height = 0.2
    speed = 0.02
    center = (position + height) / 2
    
    def __init__(self, pos):
        self.position = pos
        
    
    def up(self):
        self.position = max(self.position - self.speed, 0)
        #return self.position

    def down(self):
        self.position = min(self.position + self.speed, 0.8)
        #return self.position
    
    
    def getpos(self):
        return self.position
    
    

In [23]:
class Test_State(State):
    terminate_left = False
    terminate_right = False


    def update(self):
        global num_bounce
        global catch
        
        self.ball_x += self.velocity_x
        self.ball_y += self.velocity_y
        
        if self.ball_x >= 1:        # hitting right bound
            
            
            if self.ball_y >= self.paddle_y and self.ball_y <= self.paddle_y + 0.2:    # hitting paddle
                self.ball_x = 2 - self.ball_x
                #print("bounce")
                num_bounce += 1
#                 if curr_state not in state_tracker:
#                     #print(curr_state)
#                     state_tracker.append(curr_state)
                                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
                    
                
                
            else:    # missing paddle
                self.ball_x =  2 - self.ball_x
#                 print("ai loss")
                self.terminate_right = True

            
        if self.ball_x <= 0:     # hitting left bound
#             print("neg x pos")
            if self.ball_y >= e.position and self.ball_y <= e.position + 0.2:    # hitting paddle
                self.ball_x = - self.ball_x

                catch += 1
                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
            else:
                self.ball_x =  - self.ball_x
#                 print("enemy loss")
                self.terminate_left = True
                    
        if self.ball_y <= 0:     # hitting upper bound
            self.ball_y = -self.ball_y
            self.velocity_y = -self.velocity_y
        if self.ball_y  >= 1:    # hitting right bound
            self.ball_y = 2 - self.ball_y
            self.velocity_y = -self.velocity_y
        
        y_diff = self.ball_y - e.center

        if y_diff < 0:
            e.up()
        elif y_diff > 0:
            e.down()
        
        #print ("self.ball_x is: ", self.ball_x)
            

In [24]:

def get_up_test(current_state):
    #print("up current", match(current_state))
    new_state = Test_State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.up()
    #print("new state", match(new_state))
    return new_state

def get_down_test(current_state):
    new_state = Test_State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.down()
    return new_state

def get_no_move_test(current_state):
    new_state = Test_State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.update()
    return new_state

In [27]:
t_ = 1
num_bounce = 0
total_num_bounce = 0

catch = 0

count_ai_win = 0
count_enemy_win = 0

for i in range(1000):
    s = Test_State(0.5, 0.5, 0.03, 0.01, 0.4)
    e = enemy(0.4)
    current_state = copy.deepcopy(s)
    print("round: ", i)
    num_bounce = 0
    catch = 0
    while s.terminate_left is False and s.terminate_right is False:
        
        next_action = Q_agent_test(current_state, t_)

        if next_action == 0:
            current_state.up()
        if next_action == 1:
            current_state.down()
        if next_action == 2:
            current_state.update()
        #print("current state", match(current_state))
    t_ += 1
    
    if (s.terminate_left): 
        
        print("AI win")
        count_ai_win += 1
    elif(s.terminate_right):
        print("enemy win")
        count_enemy_win += 1
    print("bounce:", num_bounce)
    total_num_bounce += num_bounce
    print("state visited: ",np.count_nonzero(Q_table))


round:  0
AI win
bounce: 4
state visited:  14245
round:  1
AI win
bounce: 4
state visited:  14245
round:  2
AI win
bounce: 4
state visited:  14245
round:  3
AI win
bounce: 8
state visited:  14245
round:  4
AI win
bounce: 4
state visited:  14245
round:  5
AI win
bounce: 4
state visited:  14245
round:  6
AI win
bounce: 4
state visited:  14245
round:  7
AI win
bounce: 4
state visited:  14245
round:  8
AI win
bounce: 4
state visited:  14245
round:  9
enemy win
bounce: 8
state visited:  14245
round:  10
AI win
bounce: 8
state visited:  14245
round:  11
AI win
bounce: 4
state visited:  14245
round:  12
AI win
bounce: 4
state visited:  14245
round:  13
AI win
bounce: 4
state visited:  14245
round:  14
AI win
bounce: 4
state visited:  14245
round:  15
AI win
bounce: 4
state visited:  14245
round:  16
AI win
bounce: 4
state visited:  14245
round:  17
AI win
bounce: 8
state visited:  14245
round:  18
AI win
bounce: 4
state visited:  14245
round:  19
AI win
bounce: 4
state visited:  14245
round: 

AI win
bounce: 8
state visited:  14245
round:  168
enemy win
bounce: 6
state visited:  14245
round:  169
AI win
bounce: 4
state visited:  14245
round:  170
AI win
bounce: 4
state visited:  14245
round:  171
AI win
bounce: 4
state visited:  14245
round:  172
AI win
bounce: 7
state visited:  14245
round:  173
AI win
bounce: 4
state visited:  14245
round:  174
AI win
bounce: 8
state visited:  14245
round:  175
AI win
bounce: 4
state visited:  14245
round:  176
AI win
bounce: 4
state visited:  14245
round:  177
AI win
bounce: 4
state visited:  14245
round:  178
AI win
bounce: 4
state visited:  14245
round:  179
enemy win
bounce: 12
state visited:  14245
round:  180
AI win
bounce: 4
state visited:  14245
round:  181
AI win
bounce: 8
state visited:  14245
round:  182
AI win
bounce: 4
state visited:  14245
round:  183
AI win
bounce: 4
state visited:  14245
round:  184
AI win
bounce: 4
state visited:  14245
round:  185
AI win
bounce: 8
state visited:  14245
round:  186
AI win
bounce: 4
state v

AI win
bounce: 4
state visited:  14245
round:  339
AI win
bounce: 4
state visited:  14245
round:  340
enemy win
bounce: 5
state visited:  14245
round:  341
AI win
bounce: 7
state visited:  14245
round:  342
AI win
bounce: 4
state visited:  14245
round:  343
AI win
bounce: 4
state visited:  14245
round:  344
AI win
bounce: 4
state visited:  14245
round:  345
AI win
bounce: 4
state visited:  14245
round:  346
AI win
bounce: 4
state visited:  14245
round:  347
AI win
bounce: 4
state visited:  14245
round:  348
AI win
bounce: 4
state visited:  14245
round:  349
AI win
bounce: 4
state visited:  14245
round:  350
AI win
bounce: 4
state visited:  14245
round:  351
AI win
bounce: 8
state visited:  14245
round:  352
AI win
bounce: 4
state visited:  14245
round:  353
AI win
bounce: 4
state visited:  14245
round:  354
AI win
bounce: 4
state visited:  14245
round:  355
AI win
bounce: 4
state visited:  14245
round:  356
AI win
bounce: 4
state visited:  14245
round:  357
AI win
bounce: 4
state visit

AI win
bounce: 4
state visited:  14245
round:  510
AI win
bounce: 8
state visited:  14245
round:  511
AI win
bounce: 12
state visited:  14245
round:  512
AI win
bounce: 4
state visited:  14245
round:  513
AI win
bounce: 4
state visited:  14245
round:  514
AI win
bounce: 8
state visited:  14245
round:  515
AI win
bounce: 4
state visited:  14245
round:  516
AI win
bounce: 8
state visited:  14245
round:  517
AI win
bounce: 7
state visited:  14245
round:  518
enemy win
bounce: 5
state visited:  14245
round:  519
AI win
bounce: 4
state visited:  14245
round:  520
AI win
bounce: 4
state visited:  14245
round:  521
enemy win
bounce: 4
state visited:  14245
round:  522
enemy win
bounce: 5
state visited:  14245
round:  523
AI win
bounce: 12
state visited:  14245
round:  524
AI win
bounce: 4
state visited:  14245
round:  525
AI win
bounce: 7
state visited:  14245
round:  526
enemy win
bounce: 4
state visited:  14245
round:  527
AI win
bounce: 4
state visited:  14245
round:  528
AI win
bounce: 4


state visited:  14245
round:  700
AI win
bounce: 4
state visited:  14245
round:  701
AI win
bounce: 4
state visited:  14245
round:  702
AI win
bounce: 4
state visited:  14245
round:  703
enemy win
bounce: 5
state visited:  14245
round:  704
AI win
bounce: 4
state visited:  14245
round:  705
enemy win
bounce: 6
state visited:  14245
round:  706
AI win
bounce: 4
state visited:  14245
round:  707
AI win
bounce: 4
state visited:  14245
round:  708
AI win
bounce: 4
state visited:  14245
round:  709
AI win
bounce: 4
state visited:  14245
round:  710
AI win
bounce: 7
state visited:  14245
round:  711
AI win
bounce: 4
state visited:  14245
round:  712
AI win
bounce: 4
state visited:  14245
round:  713
AI win
bounce: 4
state visited:  14245
round:  714
AI win
bounce: 4
state visited:  14245
round:  715
enemy win
bounce: 4
state visited:  14245
round:  716
AI win
bounce: 4
state visited:  14245
round:  717
AI win
bounce: 4
state visited:  14245
round:  718
AI win
bounce: 4
state visited:  14245


AI win
bounce: 8
state visited:  14245
round:  874
AI win
bounce: 8
state visited:  14245
round:  875
AI win
bounce: 6
state visited:  14245
round:  876
AI win
bounce: 4
state visited:  14245
round:  877
AI win
bounce: 4
state visited:  14245
round:  878
AI win
bounce: 4
state visited:  14245
round:  879
AI win
bounce: 4
state visited:  14245
round:  880
AI win
bounce: 4
state visited:  14245
round:  881
AI win
bounce: 4
state visited:  14245
round:  882
AI win
bounce: 4
state visited:  14245
round:  883
AI win
bounce: 4
state visited:  14245
round:  884
AI win
bounce: 4
state visited:  14245
round:  885
AI win
bounce: 7
state visited:  14245
round:  886
AI win
bounce: 4
state visited:  14245
round:  887
AI win
bounce: 4
state visited:  14245
round:  888
AI win
bounce: 4
state visited:  14245
round:  889
AI win
bounce: 4
state visited:  14245
round:  890
enemy win
bounce: 12
state visited:  14245
round:  891
AI win
bounce: 4
state visited:  14245
round:  892
AI win
bounce: 4
state visi

In [26]:
print("average bouncing per game: ", total_num_bounce / 1000)

print("totoal state visited after training: ", np.count_nonzero(Q_table))

average bouncing per game:  5.236
totoal state visited after training:  14245


In [28]:
print("ai win times is: ", count_ai_win)
print("enemy win times is: ", count_enemy_win)


ai win times is:  917
enemy win times is:  83
